In [50]:
#hide
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
from load_covid_data import load_individual_timeseries
import utils
%config InlineBackend.figure_format = 'retina'

chart_width = 800
chart_height= 720

In [51]:
Since reaching at least 1 death per million

> Tip: Click (Shift+ for multiple) on countries in the legend to filter the visualization.

SyntaxError: invalid syntax (<ipython-input-51-26a600a95de1>, line 1)

In [52]:
#hide 
cdata = utils.getComulativeData('confirmed')
data = utils.getComulativeData('deaths')
data = data.rename(columns={"total_deaths": "total_cases"})

countries=pd.Series(data.location.unique())


In [53]:
#hide
# Population data
data_pop=pd.read_csv("data_files/world_population.csv")[["country","population"]]
data_pop=data_pop[data_pop.country.isin(countries)]
# To list the countries who's population is not found in world bank
# pd.set_option('display.max_rows', None)
# countries=countries[~countries.isin(data_pop.country)]
countries=countries[countries.isin(data_pop.country)]


display(data_pop.country.unique().size , countries.size)
# (countries.isin(data_pop.country))[countries == False]

175

175

In [54]:
# data_pc=data_pc[data_pc.location == 'China']
# data_pc.total_cases.max()
# data_pop=data_pop.apply(lambda x: [(x/1000000)], 1)
# data_pop=data_pop[data_pop.country.isin(countries)]
# data_pop=data_pop[data_pop.population > 0]

data_pc=data.copy()
data_pc=data_pc[data_pc.location.isin(countries)]

# pd.set_option('display.max_rows', 10)
# data_pc[data_pc.total_cases==float('nan') | float('inf')]
data_pc

,location,date,total_cases
0,Afghanistan,2020-01-22,0
12768,Mauritius,2020-01-22,0
12882,Mexico,2020-01-22,0
12996,Moldova,2020-01-22,0
13110,Monaco,2020-01-22,0
...,...,...,...
8997,Iceland,2020-05-14,10
9111,India,2020-05-14,2649
9225,Indonesia,2020-05-14,1043
2955,Bulgaria,2020-05-14,99


In [55]:
for i in countries:
    data_pc.loc[data_pc.location == i,"total_cases"] = data_pc.loc[data_pc.location == i,"total_cases"]/(data_pop.loc[data_pop.country == i, "population"].astype(float).to_list()[0]/1000000)

data_countries_pc=[]

ranked_data=pd.DataFrame(columns=['country','total_cases_per_milliion_pop'])#country, total_cases_per_milliion_pop
# ranked_data.append({'a','b'})
for i in countries:
    filter_country = data_pc["location"]== i
    temp=data_pc[filter_country]
    # display(temp)
    # if temp['total_cases'].tail(1).to_list()[0]> 500:
    if temp['total_cases'].sum() > 0 :
        ranked_data=ranked_data.append([{'country':i,'total_cases_per_milliion_pop':temp['total_cases'].tail(1).to_list()[0]}])

ranked_data=ranked_data.sort_values(by='total_cases_per_milliion_pop',ignore_index=True,ascending=False)
countries=ranked_data['country']
countries=countries[0:20]
for i in countries:
    filter_country = data_pc["location"]== i
    # temp=data_pc[filter_country]
    data_countries_pc.append(data_pc[filter_country])
# pd.set_option('display.max_rows', None)
# display(ranked_data)


In [56]:
pd.set_option('display.max_rows', 10)
len(data_countries_pc)

20

In [57]:
for i in range(0,len(countries)):
    data_countries_pc[i]=data_countries_pc[i].reset_index()
    data_countries_pc[i]['x_day'] = data_countries_pc[i].index
len(data_countries_pc)

20

In [58]:
data_plot=pd.DataFrame()
for i in range(0,len(countries)):
    data_plot = pd.concat([data_plot, data_countries_pc[i]], axis=0)
# data_plot

data_plot_death_log=data_plot.copy()
def a(x):
    if x==0:
        return 0
    return np.log(x)
pd.set_option('display.max_rows', 100)

# data_plot['total_log_cases']=data_plot['total_cases'].apply(a)
data_plot

,index,location,date,total_cases,x_day
0,16530,San Marino,2020-01-22,0.000000,0
1,16531,San Marino,2020-01-23,0.000000,1
2,16532,San Marino,2020-01-24,0.000000,2
3,16533,San Marino,2020-01-25,0.000000,3
4,16534,San Marino,2020-01-26,0.000000,4
...,...,...,...,...,...
109,9335,Iran,2020-05-10,79.054255,109
110,9336,Iran,2020-05-11,79.590014,110
111,9337,Iran,2020-05-12,80.161491,111
112,9338,Iran,2020-05-13,80.756779,112


In [59]:
# Plot it using Altair
source = data_plot

scales = alt.selection_interval(bind='scales', zoom=True)
selection = alt.selection_multi(fields=['location'], bind='legend')
alt.data_transformers.disable_max_rows()

base = alt.Chart(source, title = "20 Highest COVID-19 Deaths Per Million of Inhabitants").encode(
    x = alt.X('date:T', title = "Days passed"),
    y = alt.Y("total_cases:Q",title = "Deaths per million"),
    color = alt.Color('location:N', legend=alt.Legend(title="Country",values=countries.to_list(), labelFontSize=15, titleFontSize=17),
                     scale=alt.Scale(scheme='tableau20')),
    opacity = alt.condition(selection, alt.value(1), alt.value(0.1))
)

lines = base.mark_line().add_selection(
    scales
).add_selection(
    selection
).properties(
    width=chart_width,
    height=chart_height
)

lines = base.mark_line().add_selection(
    scales
).add_selection(
    selection
).properties(
    width=chart_width,
    height=chart_height
)
lines


alt.Chart(...)

In [60]:
# base = alt.Chart(source, title = "COVID-19 Deaths Per Million of Inhabitants").encode(
#     x = alt.X('x_day:Q', title = "Days passed"),
#     y = alt.Y("total_log_cases:Q",title = "Log of deaths per million"),
#     color = alt.Color('location:N', legend=alt.Legend(title="Country", labelFontSize=15, titleFontSize=17),
#                      scale=alt.Scale(scheme='tableau20')),
#     opacity = alt.condition(selection, alt.value(1), alt.value(0.1))
# )

# lines = base.mark_line().add_selection(
#     scales
# ).add_selection(
#     selection
# ).properties(
#     width=chart_width,
#     height=chart_height
# )

# lines = base.mark_line().add_selection(
#     scales
# ).add_selection(
#     selection
# ).properties(
#     width=chart_width,
#     height=chart_height
# )
# lines

In [61]:
df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'},
                   'B': {0: 's', 1: 's', 2: 's'},
                   'C': {0: 1, 1: 2, 2: 3},
                   'D': {0: 9, 1: 8, 2: 7}
                   })
display(df)
df=df.melt(id_vars=['A','B'])
display(df)
df = pd.DataFrame(df.groupby(['B', "variable"]).sum())
display(df)


,A,B,C,D
0,a,s,1,9
1,b,s,2,8
2,c,s,3,7


,A,B,variable,value
0,a,s,C,1
1,b,s,C,2
2,c,s,C,3
3,a,s,D,9
4,b,s,D,8
5,c,s,D,7


value
B variable       
s C             6
  D            24